In [ ]:
#I have the final results data now.. its time to turn it into LIE Data and explain what each data mean 


In [ ]:
 #currentdir: /Users/oanhtt/Documents/UCI/NOS/BindingFE/E.MMPBSA/ResultTSCC/ 
#change to  newdir: /Users/oanhtt/Documents/UCI/NOS/BindingFE/F.MMPBSA2LIE/  

def directory(initdir):
    ligfile = ["J4", "J5", "J5flip", "J9", "J9flip", "J10", "J11", "J12", "J13", "J14" ]
    nosfile = ["nNOS", "eNOS", "iNOS"]
    for lig in ligfile:
        for nos in nosfile:
            print ("%s/%s/%s" %(initdir, lig,nos))
            return "%s/%s/%s" %(initdir, lig,nos

In [2]:
#8/25/2020#makefiles:
import os 

ligfile = ["J4", "J5", "J5flip", "J9", "J9flip", "J10", "J11", "J12", "J13", "J14" ]
nosfile = ["nNOS", "eNOS", "iNOS"]
for lig in ligfile:
    #os.mkdir(lig)
    for nos in nosfile:
        #os.chdir("/Users/oanhtt/Documents/UCI/NOS/BindingFE/F.MMPBSA2LIE/%s" %(lig))
        #os.mkdir(nos)

In [12]:
#8/25/2020 #trasnfer the file over 
import shutil 
import os 
import pathlib
ligfile = ["J4", "J5", "J5flip", "J9", "J9flip", "J10", "J11", "J12", "J13", "J14" ]
nosfile = ["nNOS", "eNOS", "iNOS"]
interestfile = [ "FINAL_RESULTS_MMPBSA2.dat", "FINAL_RESULTS_MMPBSA1.dat"]
for lig in ligfile:
    for nos in nosfile:
        file = pathlib.Path("/Users/oanhtt/Documents/UCI/NOS/BindingFE/E.MMPBSA/ResultTSCC/%s/iNOS" %lig)
        if file.exists():
            for inter in interestfile:
                orig =  "/Users/oanhtt/Documents/UCI/NOS/BindingFE/E.MMPBSA/ResultTSCC/%s/%s/%s" %(lig,nos, inter)
                outputdir = "/Users/oanhtt/Documents/UCI/NOS/BindingFE/F.MMPBSA2LIE/%s/%s/%s" %(lig,nos, inter)
                try: 
                    shutil.copy(orig, outputdir)
                except: 
                    print ("error in %s/%s/%s" %(lig,nos,inter))
        else :  
            pass 
#note. J14 has its own file, it seems that file MMPBSA2 is missing for file j9/inos- will redo it in 

error in J9/iNOS/FINAL_RESULTS_MMPBSA2.dat
error in J14/iNOS/FINAL_RESULTS_MMPBSA2.dat
error in J14/iNOS/FINAL_RESULTS_MMPBSA1.dat


In [14]:
#sample from what the results look like 
with open ("/Users/oanhtt/Documents/UCI/NOS/BindingFE/F.MMPBSA2LIE/J4/nNOS/FINAL_RESULTS_MMPBSA1.dat", "r") as file1:
    rreading = file1.read() 
    print (rreading)

| Run on Sat Jul  4 08:27:11 2020
|
|Input file:
|--------------------------------------------------------------
|
|Input file for running PB and GB
|&general
|   endframe=100, keep_files=2,
|   receptor_mask=!:1, ligand_mask=:1
|/
|&gb
|  igb=2, saltcon=0.100,
|/
|&pb
|  istrng=0.100, radiopt=0
|/--------------------------------------------------------------
|MMPBSA.py Version=14.0
|Solvated complex topology file:  complex_wat_bond.prmtop
|Complex topology file:           complex1.prmtop
|Receptor topology file:          receptor1.prmtop
|Ligand topology file:            ligand1.prmtop
|Initial mdcrd(s):                nnosMC10prod.nc
|
|Receptor mask:                  "!:1"
|Ligand mask:                    ":1"
|Ligand residue name is "JI4"
|
|Calculations performed using 43.0 complex frames.
|Poisson Boltzmann calculations performed using internal PBSA solver in mmpbsa_py_energy
|
|Generalized Born ESURF calculated using 'LCPO' surface areas
|
|All units are reported in kcal/mole.
-

In [ ]:
"""#!bin/bash
echo "  "
echo "  "
echo "Useage: input_normal_mmgbsa input_mmgbsa_with_protein_charges_off  output_dGlie_file "
echo "  "
echo "Input normal mmgbsa file                                $1"
echo "Input mmgbsa file with protein charges zero in receptor $2"
echo "Output Deltag G file                                    $3"
echo "   "
#This script takes output from MMGBSA and calculates a LIE energy
# See Carlsson et al (2006) J. Phys. Chem 110, 12034
# The electrostatic terms in COM-REC + 2*LIGbound - 2*LIGfree
# COM-REC is just the difference in EEL between COM and REC. LIGbound
# is the GB reaction field solvation energy of the ligand bound to the protein.
# This is computed by setting the protein charges to zero in the MMGBSA calculation
# for the receptor. LIGfree is just the GB reaction field foe ligand alone
# VDW is just the usual COM-LIG-LIG plus GBSUR (nonpolar solvation energy).
# which is the reaction field. 
####the final dGlie= 0.50(deltatEEL + 0.18(VdeltatDW)
####complex-receptor-ligand EEL#######
grep ELE $1 > tempa
sed '2,4d' tempa > tempb
awk '{dif=$2-$4} END {print dif}' tempb > tempELEcom-rec
sed '1,2d' tempa > tempb
sed '$d' tempb > tempc
awk '{print $2}' tempc > temp_com-rec_EEL
####get the GB for ligand bound to protein########
grep "GB " $2 > tempa
sed '2d' tempa > tempb
#awk '{dif=$2-$4} END {print dif}' tempb > tempGBligand_bound
awk '{print $4}' tempb > tempGBligand_bound
############get the GB for the ligand alone##########
grep "GB " $1  > tempa
sed '2d' tempa > tempb
awk '{print $6}' tempb > tempGBligand_free
###########aseemble electrostatic part###########
paste tempELEcom-rec tempGBligand_bound tempGBligand_free >  tempEELall
awk '{sum=$1+2*$2-2*$3} END {print sum}' tempEELall > tempDeltaEEL
###################get VDW and GBSUR terms###########
grep VDW $1 > tempa
sed '1d' tempa > tempb
awk '{print $2}' tempb > tempDeltaVDW
grep GBSUR $1 > tempa
sed '1d' tempa > tempb
awk '{print $2}' tempb > tempDeltaGBSUR
###########assemble VDW and GBSUR########
paste tempDeltaVDW tempDeltaGBSUR> tempDeltatVDW_GBUSR
awk '{sum = $1+$2} END {print sum}' tempDeltatVDW_GBUSR > tempDeltatVDW+GBSUR
###########assemble EEL and VDW for final calculation##################
paste tempDeltaEEL tempDeltatVDW+GBSUR > tempa
awk '{sum = 0.50*$1 + 0.18*$2}; END {print "dGlie = " sum}' tempa > $3 
rm temp*"""

-------
## This is from MMGBSA2LIE from poulos 
---------

In [15]:
with open ("/Users/oanhtt/Documents/UCI/NOS/BindingFE/F.MMPBSA2LIE/mmpbsa2lie.sh", "r") as file2:
    mmpbsa2lie = file2.read() 
    print (mmpbsa2lie)


#!bin/bash
echo "  "
echo "  "
echo "Useage: input_normal_mmgbsa input_mmgbsa_with_protein_charges_off  output_dGlie_file "
echo "  "
echo "Input normal mmgbsa file                                $1"
echo "Input mmgbsa file with protein charges zero in receptor $2"
echo "Output Deltag G file                                    $3"
echo "   "
#This script takes output from MMGBSA and calculates a LIE energy
# See Carlsson et al (2006) J. Phys. Chem 110, 12034
# The electrostatic terms in COM-REC + 2*LIGbound - 2*LIGfree
# COM-REC is just the difference in EEL between COM and REC. LIGbound
# is the GB reaction field solvation energy of the ligand bound to the protein.
# This is computed by setting the protein charges to zero in the MMGBSA calculation
# for the receptor. LIGfree is just the GB reaction field foe ligand alone
# VDW is just the usual COM-LIG-LIG plus GBSUR (nonpolar solvation energy).
# which is the reaction field. 
####the final dGlie= 0.50(deltatEEL + 0.18(VdeltatDW)
####c